In [1]:
from __future__ import division, print_function
# coding=utf-8
import sys
import os
import glob
import re
import numpy as np

# Keras
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer

# Define a flask app



In [2]:
app = Flask(__name__)

# Model saved with Keras model.save()
MODEL_PATH = 'model6.h5'

# Load your trained model
model = load_model(MODEL_PATH)
model.make_predict_function()          

print('Model loaded. Check http://127.0.0.1:5000/')


def model_predict(img_path, model):
    
    
    dict = { 0 : '૦', 1 : '૧', 2 : 'જૂ', 3 : 'જે', 4 : 'જૈ', 5 :'જો', 6 : 'જૌ', 7 : 'જં',8 : 'જઃ', 9 : 'ઝ', 10 : 'ઝા', 11 : 'ઝિ', 12 : 'અ', 13 : 'ઝી',
      14 : 'ઝુ', 15 : 'ઝૂ', 16 : 'ઝે', 17 : 'ઝૈ', 18 : 'ઝો', 19 : 'ઝૌ', 20 : 'ઝં', 21 : 'ઝઃ', 22 : 'ટ', 23 : 'આ', 24 : 'ટા',25 : 'ટિ', 26 : 'ટી', 
      27 : 'ટુ', 28 : 'ટૂ', 29 : 'ટે', 30 : 'ટૈ', 31 : 'ટો', 32 : 'ટૌ', 33 : 'ટં', 34 : 'ઇ', 35 : 'ટઃ', 36 : 'ઠ' ,37 : 'ઠા', 38 :'ઠિ', 39 :'ઠી', 
      40 : 'ઠુ', 41 : 'ઠૂ', 42 : 'ઠે',43 : 'ઠૈ', 44 : 'ઠો',45 : 'ઈ', 46 : 'ઠૌ', 47 : 'ઠં', 48 : 'ઠઃ', 49 : 'ડ', 50 : 'ડા', 51 : 'ડિ',52 : 'ડી', 
      53 : 'ડુ', 54 : 'ડૂ', 55 : 'ડે', 56 : 'ઉ', 57 : 'ડૈ',58 : 'ડો',59 : 'ડૌ', 60 : 'ડં', 61 : 'ડઃ', 62 : 'ઢ', 63 :'ઢા', 64 : 'ઢિ', 65 : 'ઢી', 66 : 'ઢુ', 
      67 : 'ઊ', 68 : 'ઢૂ', 69:  'ઢે', 70 : 'ઢૈ', 71 : 'ઢો',72 : 'ઢૌ',73 : 'ઢં',74 : 'ઢઃ',75 : 'ણ',76 : 'ણા', 77 : 'ણિ', 78 : 'ઋ',79 : 'ણી', 
      80 : 'ણુ', 81 : 'ણૂ',82 : 'ણે',83 : 'ણૈ', 84 : 'ણો', 85 : 'ણૌ', 86 : 'ણં', 87 :'ણઃ', 88 : 'ત',89 : 'એ', 90 : 'તા', 91 :'તિ', 92 :'તી', 93 : 'તુ', 
      94 : 'તૂ', 95 : 'તે', 96 : 'તૈ', 97 : 'તો', 98 : 'તૌ', 99 : 'થ', 100 : 'ઐ', 101 : 'થા', 102 : 'થિ',103 : 'થી', 104 : 'થુ', 105 : 'થૂ', 
      106 : 'થે',107 : 'થૈ',108 : 'થો',109 : 'થૌ',110 : 'થં', 111 : 'ઓ', 112 :'2', 113 :'થઃ',114 : 'દ',115: 'દા',116 : 'દિ', 117 : 'દી', 118 : 'દુ', 
      119 : 'દૂ', 120 : 'દે', 121 : 'દૈ',122: 'દો', 123: 'ઔ',124 : 'દૌ',125: 'દં',126 : 'દઃ',127 : 'ધ',128 : 'ધા',129: 'ધિ',130 : 'ધી', 131 : 'ધુ',
      132 :'ધૂ', 133 : 'ધે', 134 : 'અં', 135 : 'ધૈ', 136:  'ધો',137 : 'ધૌ', 138: 'ધં', 139 :'ધઃ',140 : 'ન', 141 : 'ના',142: 'નિ',143: 'ની', 144 : 'નુ', 
      145 : 'અઃ',146 : 'નૂ', 147: 'ને',148 : 'નૈ', 149 :'નો',150: 'નૌ',151: 'નં',152 : 'નઃ', 153 :'પ',154: 'પા', 155 :'પિ', 156 :'ક', 157:  'પી', 
      158 :'પુ', 159 :'પૂ', 160 : 'પે',161: 'પૈ',162: 'પો',163: 'પૌ',164 : 'પં',165 : 'પઃ',166: 'ફ',167: 'કા',168: 'ફા',169: 'ફિ',170: 'ફી',171 : 'ફુ',
      172 :'ફૂ',173 : 'ફે',174 : 'ફૈ',175 : 'ફો',176 : 'ફૌ',177 : 'ફં', 178 : 'કિ', 179 : 'ફઃ',180 : 'બ', 181 :'બા', 182 : 'બિ', 183 : 'બી', 184 :'બુ',
      185 :'બૂ', 186 : 'બે',187 : 'બૈ',188 : 'બો',189 : 'કી', 190 : 'બૌ',191 : 'બં',192: 'બઃ', 193 : 'ભ',194 : 'ભા', 195 : 'ભિ', 196 : 'ભી',
      197 :'ભુ', 198 : 'ભૂ', 199 :'ભે',200 : 'કુ', 201 :'ભૈ',202 : 'ભો', 203 : 'ભૌ',204 : 'ભં',205 : 'ભઃ',206: 'મ',207 : 'મા', 208 : 'મિ', 209 : 'મી',
      210 : 'મુ', 211 : 'કૂ',212 : 'મૂ',213 : 'મે',214 : 'મૈ',215 : 'મો',216 : 'મૌ',217 : 'મં',218 : 'મઃ', 219 : 'ય',220 : 'યા',221 : 'યિ',222 : 'કે', 
      223 : '૩', 224 : 'યી',225 : 'યુ',226 : 'યૂ',227 : 'યે',228 : 'યૈ',229 : 'યો',230 : 'યૌ',231 : 'યં', 232 : 'યઃ',233 : 'ર',234 : 'કૈ', 235 : 'રા', 
      236 : 'રિ',237 : 'રી',238 : 'રુ',239 : 'રૂ',240 : 'રે',241 : 'રૈ',242 : 'રો',243 : 'રૌ',244 : 'રં',245 : 'કો',246 : 'રઃ',247 : 'લ',248 : 'લા', 249 :'લિ', 
      250 : 'લી',251 : 'લુ',252 : 'લૂ',253 : 'લે',254 : 'લૈ',255 : 'લો',256 : 'કૌ',257 : 'લૌ', 258: 'લં',259 : 'લઃ',260 : 'વ',261 : 'વા', 262 :'વિ',263 : 'વી', 
      264 :'વુ', 265 : 'વૂ',266 : 'વે',267 : 'કં',268 :'વૈ', 269 : 'વો', 270 : 'વૌ', 271 : 'વં',272: 'વઃ',273 : 'શ',274 : 'શા', 275 : 'શિ', 276 : 'શી', 
      277 : 'શુ',278 : 'કઃ', 279:  'શૂ',280 : 'શે',281 : 'શૈ',282 : 'શો',283: 'શૌ',284: 'શં',285 : 'શઃ',286 : 'ષ', 287 : 'ષા',288: 'ષિ',289 : 'ખ', 
      290 : 'ષી', 291 : 'ષુ',292 : 'ષૂ',293 : 'ષે',294 : 'ષૈ',295: 'ષો',296 : 'ષૌ',297: 'ષં',298: 'ષઃ',299 : 'સ',300: 'ખા',301 : 'સા', 302 : 'સિ',303 : 'સી', 
      304 :'સુ',305 : 'સૂ', 306 : 'સે',307 : 'સૈ',308 : 'સો',309 : 'સૌ',310 : 'સં',311 : 'ખિ',312 : 'સઃ',313 : 'હ',314 : 'હા',315 : 'હિ',316 : 'હી', 
      317 : 'હુ',318 : 'હૂ', 319 : 'હે',320 : 'હૈ',321 : 'હો',322: 'ખી',323: 'હૌ',324: 'હં',325 : 'હઃ',326 : 'ળ',327 : 'ળા',328 : 'ળિ',329 : 'ળી',330 : 'ળુ',
      331 : 'ળૂ', 332 : 'ળે', 333 : 'ખુ', 334 : '૪', 335 :'ળૈ',336 : 'ળો',337 : 'ળૌ',338 : 'ળં',339 : 'ળઃ',340 : 'ક્ષ',341 : 'ક્ષા',342 : 'ક્ષિ',343 : 'ક્ષી', 
      344 :'ક્ષુ', 345 :'ખૂ',346 : 'ક્ષૂ', 347 :'ક્ષે', 348 : 'ક્ષૈ' ,348 : 'ક્ષૈ',349 : 'ક્ષો',350 : 'ષૌ',351 : 'ક્ષં',352 : 'ક્ષઃ',353 : 'જ્ઞ',354 : 'જ્ઞા',355 : 'જ્ઞિ',356 : 'ખે', 
      357 :'જ્ઞી',358 : 'જ્ઞુ',359 : 'જ્ઞૂ', 360 : 'જ્ઞે',361 : 'જ્ઞૈ',362 : 'જ્ઞો',363 : 'જ્ઞૌ',364 : 'જ્ઞં',365 : 'જ્ઞઃ',366 : 'કૃ', 367 :'ખૈ',368 : 'ઞ',369 : 'ત્ર', 370 : 'ત્રા', 
      371 :'ત્રિ',372 : 'ત્રી',373 : 'ત્રુ',374 : 'ત્રૂ',375 : 'ત્રે',376 : 'ત્રો',377 : 'ત્રૈ',378 : 'ખો',379 : 'ત્રૌ',380 : 'દ્ર',381 : 'શ્રા',382 : 'શ્ર',383 : 'તં',384 : 'તઃ', 
      385 :'ખૌ',386 : 'ખં',387 : 'ખઃ',388 : 'ગ',389 : 'ગા',390 : 'ગિ',391 : '૫',392 : 'ગી',393 : 'ગુ',394 : 'ગૂ',395 : 'ગે',396 : 'ગૈ',397 : 'ગો', 398 :'ગૌ', 
      399 : 'ગં',400 : 'ગઃ',401 : 'ઘ',402 : '૬',403 : 'ઘા',404 : 'ઘિ',405 : 'ઘી',406 : 'ઘુ',407 : 'ઘૂ',408 : 'ઘે', 409 :'ઘૈ',410 : 'ઘો',411: 'ઘૌ',412 : 'ઘં', 
      413 :'૭', 414 : 'ઘઃ', 415 : 'ચ', 416 : 'ચા',417 : 'ચિ',418 : 'ચી',419 : 'ચુ', 420 :'ચૂ', 421 :'ચે', 422 : 'ચૈ',423 : 'ચો',424 : '૮',425 : 'ચૌ',426 : 'ચં', 
      427 :'ચઃ',428 : 'છ',429 : 'છા',430 : 'છિ',431 : 'છી',432 : 'છુ',433: 'છૂ',434: 'છે',435: '૯',436: 'છૈ',437 : 'છો',438 : 'છૌ',439 : 'છં',440 : 'છઃ', 
      441: 'જ',442:  'જા',443 : 'જિ',444: 'જી',445 : 'જુ'}
    
    
    img = image.load_img(img_path,color_mode = "grayscale", target_size=(28, 28))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
   # x = img.reshape(32, 32)
    prediction = model.predict(img.reshape(1,28,28, 1))
    #pred = model.predict(img)
    print(prediction)
    return str(dict[np.argmax(prediction)])


@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        basepath = os.path.dirname('__file__')
        file_path = os.path.join(
            basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)

        # Make prediction
        result = model_predict(file_path, model)
        return result
    return None
    



Model loaded. Check http://127.0.0.1:5000/


In [ ]:

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Dec/2021 18:33:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 18:33:04] "GET /assets/vendor/aos/aos.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Dec/2021 18:33:04] "GET /assets/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Dec/2021 18:33:04] "GET /assets/vendor/boxicons/css/boxicons.min.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Dec/2021 18:33:04] "GET /assets/vendor/swiper/swiper-bundle.min.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Dec/2021 18:33:04] "GET /assets/vendor/bootstrap-icons/bootstrap-icons.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Dec/2021 18:33:04] "GET /assets/vendor/glightbox/css/glightbox.min.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Dec/2021 18:33:04] "GET /assets/vendor/aos/aos.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Dec/2021 18:33:48] "POST /predict HTTP/1.1" 200 -


[[5.74595338e-25 1.20250473e-22 1.14991079e-12 7.42586423e-18
  8.93334018e-12 2.30759754e-18 1.31809880e-14 8.16652822e-12
  8.49700169e-15 3.81204970e-08 8.37763636e-09 1.74592684e-11
  3.27876198e-11 2.59470792e-08 3.07029353e-07 6.57826092e-07
  4.04437206e-10 4.72547832e-08 7.05928718e-11 9.93616966e-10
  3.57351393e-09 2.41797915e-09 5.28810556e-20 1.58925526e-13
  1.42026498e-19 1.92463706e-19 2.79279104e-17 3.73993981e-14
  7.94863286e-19 2.43624775e-19 8.01859555e-17 2.75006824e-18
  3.18763082e-17 6.53917178e-17 2.81702893e-15 3.38299806e-21
  2.63053131e-23 5.25241107e-16 6.19288488e-14 1.12873453e-16
  2.38256686e-17 3.75670253e-19 1.68481618e-22 2.36496496e-16
  5.07754676e-23 6.92671516e-17 5.86398999e-21 1.27246310e-17
  1.51455982e-19 3.32685028e-16 2.41412116e-17 6.43236346e-14
  3.58980457e-14 3.16660950e-13 1.05750475e-15 3.78704998e-18
  1.10316080e-14 9.28368285e-11 2.24160669e-17 8.70065975e-15
  3.88694552e-16 1.00304128e-15 5.05939026e-16 5.44699783e-15
  4.1802

127.0.0.1 - - [07/Dec/2021 18:34:00] "POST /predict HTTP/1.1" 200 -


[[1.45066696e-27 1.87860723e-26 2.19103245e-20 3.00267623e-22
  1.20280678e-14 4.73968085e-16 3.14345011e-17 5.18791745e-16
  1.28234368e-10 7.34586922e-22 1.06099945e-18 4.47426308e-15
  2.02891581e-15 1.82922774e-16 7.27726826e-19 2.43019146e-17
  1.48957146e-22 7.29370838e-15 9.75385723e-20 1.55040613e-13
  2.43011974e-15 3.02075648e-07 3.66269896e-29 5.10502919e-17
  7.23770959e-18 3.03474117e-14 5.78394734e-22 3.15642609e-18
  1.30348300e-20 2.33974943e-18 7.66411226e-17 7.19032505e-17
  7.33452341e-16 1.05025760e-22 8.13280757e-23 2.85649733e-11
  1.79751115e-32 9.71958313e-18 9.53023541e-20 1.76375541e-22
  4.13122758e-26 9.11843118e-30 2.07742956e-27 2.95173677e-17
  6.53116245e-21 1.45354907e-17 8.00194207e-19 1.98663317e-21
  1.02182867e-12 3.69055483e-28 5.72308939e-21 1.22680769e-15
  4.96783415e-20 3.49751318e-25 5.60618873e-21 1.38902402e-18
  4.68451701e-23 6.59522892e-14 1.16418188e-19 7.20946604e-14
  1.13544877e-20 6.53693766e-12 1.18144643e-19 1.13692803e-14
  6.1092